In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, LSTM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle
import numpy as np

In [2]:
# Load the reviews and labels from the file
with open('train_reviews.pkl', 'rb') as f:
    train_reviews, train_labels = pickle.load(f)

with open('test_reviews.pkl', 'rb') as f:
    test_reviews, test_labels = pickle.load(f)

In [3]:
# Load the tokenized and padded sequences
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')

# Load the one-hot encoded labels
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

# Load the tokenizer
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

max_words = 10000
max_review_length = 500

In [4]:
# Define the LSTM model architecture
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(max_words, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(10, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=64)

# Save the model
model.save('LSTM_model.keras')

C:\Users\DELL\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 191s 480ms/step - accuracy: 0.6132 - loss: 0.6204
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 200s 510ms/step - accuracy: 0.8921 - loss: 0.2712
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 202s 518ms/step - accuracy: 0.9228 - loss: 0.2053
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 210s 537ms/step - accuracy: 0.9418 - loss: 0.1611
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 220s 562ms/step - accuracy: 0.9569 - loss: 0.1239


In [5]:
# Load the model
model = load_model('LSTM_model.keras')

In [6]:
# Predict the test set
y_pred = model.predict(x_test)

# Convert predictions to class labels
y_pred_labels = [1 if pred[1] > pred[0] else 0 for pred in y_pred]

# Calculate and print accuracy, precision, recall, and F1 score
accuracy = accuracy_score(test_labels, y_pred_labels)
precision = precision_score(test_labels, y_pred_labels)
recall = recall_score(test_labels, y_pred_labels)
f1 = f1_score(test_labels, y_pred_labels)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 117s 149ms/step
Accuracy: 0.86
Precision: 0.87
Recall: 0.85
F1 Score: 0.86


In [12]:
# Take user input for the review
input_review = input("Enter a review: ")

# Preprocessing function
def preprocess_review(review, tokenizer, maxlen):
    # Convert the review to a sequence of integers
    sequences = tokenizer.texts_to_sequences([review])
    # Pad the sequences to ensure uniform input length
    padded_sequences = pad_sequences(sequences, maxlen=maxlen)
    return padded_sequences

# Preprocess the input review
preprocessed_review = preprocess_review(input_review, tokenizer, max_review_length)

# Predict the sentiment of the input review
y_pred = model.predict(preprocessed_review)

# Convert prediction to class label (1 for positive, 0 for negative)
y_pred_label = 1 if y_pred[0][1] > y_pred[0][0] else 0

# Print the predicted sentiment
sentiment = "positive" if y_pred_label == 1 else "negative"
print(f"The predicted sentiment for the review is: {sentiment}")

Enter a review: If you had asked me how the movie was throughout the film, I would have told you it was great! However, I left the theatre feeling unsatisfied. After thinking a little about it, I believe the problem was the pace of the ending. I feel that the majority of the movie moved kind of slow, and then the ending developed very fast. So, I would say the ending left me disappointed.<br /><br />I thought that the characters were well developed. Costner and Kutcher both portrayed their roles very well. Yes! Ashton Kutcher can act! Also, the different relationships between the characters seemed very real. Furthermore,I thought that the different plot lines were well developed. Overall, it was a good movie and I would recommend seeing it.<br /><br />In conclusion: Good Characters, Great Plot, Poorly Written/Edited Ending. Still, Go See It!!!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
The predicted sentiment for the review is: positive
